# CS 584 :: Data Mining :: George Mason University :: Fall 2025


# Homework 2: Linear Regression&Neural Networks

- **100 points [6% of your final grade]**
- **Due Sunday, October 19 by 11:59pm**

- *Goals of this homework:* (1) implement the linear regression model; (2) implement the multi-layer perceptron neural network; (3) tune the hyperparameters of MLP model to produce classification result as good as possible.

- *Submission instructions:* for this homework, you should submit your notebook file to **Canvas** (look for the homework 2 assignment there). Please name your submission **FirstName_Lastname_hw2.ipynb**, so for example, my submission would be something like **Ziwei_Zhu_hw2.ipynb**. Your notebook should be fully executed so that we can see all outputs.

## Part 1: Linear Regression (40 points)

Recent studies have found that novel mobile games can lead to increased physical activity. A notable example is Pokemon Go, a mobile game combining the Pokemon world through augmented reality with the real world requiring players to physically move around. Specifically, in the following study, researchers have found that Pokemon Go leads to increased levels of physical activity for the most engaged players! https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5174727/
![image.png](attachment:image.png)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In this part, our goal is to predict the combat point of each pokemon in the 2017 Pokemon Go mobile game. Each pokemon has its own unique attributes that can help predicting its combat points. These include:

- Stamina
- Attack value
- Defense value
- Capture rate
- Flee rate
- Spawn chance
- Primary strength

The file pokemon_data.csv contains data of 146 pokemons to be used in this homework. The rows of these files refer to the data samples (i.e., pokemon samples), while the columns denote the name of the pokemon (column 1), its attributes (columns 2-8), and the combat point outcome (column 9). You can ignore column 1 for the rest of this problem.

First, let's load the data by executing the following code.

**Note: you need to install the pandas library beforehand**

In [ ]:
import numpy as np
import pandas as pd

data_frame = pd.read_csv('/content/drive/MyDrive/hw2/pokemon_data.csv')
data_frame.head()

,name,stamina,attack_value,defense_value,capture_rate,flee_rate,spawn_chance,primary_strength,combat_point
0,Bulbasaur,90,126,126,0.16,0.10,69.0,Grass,1079
1,Ivysaur,120,156,158,0.08,0.07,4.2,Grass,1643
2,Venusaur,160,198,200,0.04,0.05,1.7,Grass,2598
3,Charmander,78,128,108,0.16,0.10,25.3,Fire,962
4,Charmeleon,116,160,140,0.08,0.07,1.2,Fire,1568


Pandas is a fast, powerful, flexible and easy-to-use open-source data analysis and manipulation tool, built on top of Python. By executing the following code, let's create one Numpy array to contain the feature data without the name column and one array to contain the combat point ground truth.

In [ ]:
features = data_frame.values[:, 1:-1]
labels = data_frame.values[:, -1]
print('array of labels: shape ' + str(np.shape(labels)))
print('array of feature matrix: shape ' + str(np.shape(features)))

array of labels: shape (146,)
array of feature matrix: shape (146, 7)


Now, you may find out that we have a categorical feature 'primary_strength' in our data. Categorical features require special attention because usually they cannot be the input of regression models as they are. A potential way to treat categorical features is to simply convert each value of the feature to a separate number. However, this might impute non-existent relative associations between the features, which might not always be representative of the data (e.g., if we assign “1” to the value “green” and “2” to the value “red”, the regression algorithm will assume that “red” is greater than “green,” which is not necessarily the case). For this reason, we can use a “one hot encoding” to represent categorical features. According to this, we will create a binary column for each category of the categorical feature, which will take a value of 1 if the sample belongs to that category, and 0 otherwise. For each categorical feature of the problem, count the number of different values and implement the one hot encoding. For the remaining of the problem, you will be working with the one hot encoding of the categorical features.


In the next cell, write your code to replace the categorical feature 'primary_strength' with **one-hot encoding** and generate the new version of the Numpy array 'features'.

**Hint: if you don't remember one hot encoding, review the slides of our first-week lecture.**

**Note: do not use sklearn to automatically generate one hot encoding.**

In [ ]:
# --- One-Hot Encoding for 'primary_strength' (no sklearn) ---

# extract the categorical column
primary_strength = features[:, -1]  # last column

# find unique categories
unique_strengths = np.unique(primary_strength)
print("Unique values in 'primary_strength':", unique_strengths)

# create an empty one-hot matrix
one_hot = np.zeros((len(primary_strength), len(unique_strengths)))

# fill one-hot matrix
for i, val in enumerate(primary_strength):
    idx = np.where(unique_strengths == val)[0][0]
    one_hot[i, idx] = 1

# remove the original categorical column
features_numeric = features[:, :-1]

# combine numeric and one-hot encoded columns
features = np.hstack((features_numeric, one_hot))


print("New features shape after encoding:", features.shape)


Unique values in 'primary_strength': ['Bug' 'Dragon' 'Electric' 'Fairy' 'Fighting' 'Fire' 'Ghost' 'Grass'
 'Ground' 'Ice' 'Normal' 'Poison' 'Psychic' 'Rock' 'Water']
New features shape after encoding: (146, 21)


Besides, you may also notice that other features have different scales. So, you need to standardize them: $({x-\mu})/{\sigma}$, where $\mu$ is the mean and $\sigma$ is the standard deviation. Write your code below.

**Hint: details about feature standardization is also in slides of our first-week lecture.**

**Note: You do not need to do standardize for one-hot encodings.**

In [ ]:
# Write your code here
# --- STANDARDIZATION OF NUMERIC FEATURES ONLY ---
''' When we “standardize” data, we make all numbers fair to compare.
Some features (like attack power) might have big numbers, while others (like flee rate) are tiny decimals.
If we don’t fix this, the big numbers can dominate the model and confuse it.
It’s like making all features “speak the same language” before giving them to the model.

So instead of:

“Attack is in hundreds, spawn chance is in decimals, stamina is in tens…”

After standardization:

“Everything is now on the same 0–1 or –1 to +1 scale.” '''

# --- STANDARDIZATION OF NUMERIC FEATURES ONLY ---

import numpy as np
import pandas as pd

# how many numeric columns we have before one-hot columns
# (in pokemon_data.csv, first 6 columns are numeric)
num_numeric = 6

# separate numeric and one-hot parts
numeric_part = features[:, :num_numeric].astype(float)
one_hot_part = features[:, num_numeric:]

# calculate mean (μ) and standard deviation (σ) for each numeric column
mean = np.mean(numeric_part, axis=0)
std = np.std(numeric_part, axis=0)

# to avoid division by zero, replace any zero std with 1
std[std == 0] = 1

# apply standardization formula: (x - μ) / σ
numeric_standardized = (numeric_part - mean) / std

# combine back standardized numeric + original one-hot columns
features = np.hstack((numeric_standardized, one_hot_part))

# print mean and std info
print("Mean of numeric columns:\n", mean)
print("\nStandard deviation of numeric columns:\n", std)
print("\nNew shape of standardized features:", features.shape)

# --- PRINT COLUMN NAMES AND FIRST 5 ROWS AFTER STANDARDIZATION ---

# define the first 6 numeric column names (based on pokemon_data.csv)
numeric_cols = ["stamina", "attack", "defense", "capture_rate", "flee_rate", "spawn_chance"]

# for the one-hot columns, use the category names from before
# (make sure you ran the one-hot encoding cell that defined `unique_strengths`)
one_hot_cols = [f"primary_strength_{str(s)}" for s in unique_strengths]

# combine all column names
all_columns = numeric_cols + one_hot_cols

# create a DataFrame to preview the features
features_df = pd.DataFrame(features, columns=all_columns)

print("\n=== Columns after standardization ===")
print(features_df.columns.tolist())

print("\n=== First 5 rows (after standardization) ===")
print(features_df.head())


Mean of numeric columns:
 [1.26301370e+02 1.48273973e+02 1.44178082e+02 2.30958904e-01
 9.45205479e-02 7.57054110e+01]

Standard deviation of numeric columns:
 [5.67191314e+01 4.02646248e+01 3.77648973e+01 1.22129107e-01
 8.21632371e-02 1.97820306e+02]

New shape of standardized features: (146, 21)

=== Columns after standardization ===
['stamina', 'attack', 'defense', 'capture_rate', 'flee_rate', 'spawn_chance', 'primary_strength_Bug', 'primary_strength_Dragon', 'primary_strength_Electric', 'primary_strength_Fairy', 'primary_strength_Fighting', 'primary_strength_Fire', 'primary_strength_Ghost', 'primary_strength_Grass', 'primary_strength_Ground', 'primary_strength_Ice', 'primary_strength_Normal', 'primary_strength_Poison', 'primary_strength_Psychic', 'primary_strength_Rock', 'primary_strength_Water']

=== First 5 rows (after standardization) ===
    stamina    attack   defense capture_rate flee_rate spawn_chance  \
0  -0.64002  -0.55319 -0.481349    -0.581015   0.06669    -0.033896   

Now, in the next cell, you need to implement your own **linear regression model** using the **Ordinary Least Squares (OLS)** solution **without regularization**. And here, you should adopt the **5-fold cross-validation** method. For each fold compute and print out the **square root** of the residual sum of squares error (RSS) between the actual and predicted outcome variable. Also compute and print out the **average** square root of the RSS over all folds.

**Note: You should implement the algorithm by yourself. You are NOT allowed to use Machine Learning libraries like Sklearn.**

**Hint: Use numpy.linalg.pinv() for calculating the inverse of a matrix.**

**Hint: details about cross-validation is in slides of KNN lecture.**

**Hint: the sqrt_RSS for each fold should be 0~3000**


In [ ]:
# Write your code here
# --- OLS LINEAR REGRESSION WITH 5-FOLD CROSS-VALIDATION (no sklearn) ---
# Assumes:
#   - `features` is your final feature matrix (one-hot done; numeric optionally standardized)
#   - `labels` is your target vector
# Outputs:
#   - sqrt_RSS per fold and the average sqrt_RSS

import numpy as np

X = features.astype(float)
y = labels.astype(float).ravel()  # ensure 1D

# Add bias (intercept) column helper
def add_bias(A):
    return np.hstack([np.ones((A.shape[0], 1)), A])

# Make 5 folds (shuffled, reproducible)
rng = np.random.default_rng(42)
idx = np.arange(X.shape[0])
rng.shuffle(idx)
folds = np.array_split(idx, 5)

sqrt_rss_per_fold = []

for k in range(5):
    val_idx = folds[k]
    train_idx = np.concatenate([folds[i] for i in range(5) if i != k])

    X_train, y_train = X[train_idx], y[train_idx]
    X_val,   y_val   = X[val_idx],   y[val_idx]

    # Add bias
    X_train_b = add_bias(X_train)
    X_val_b   = add_bias(X_val)

    # OLS with pseudo-inverse: theta = pinv(X) y  (equivalently pinv(X^T X) X^T y)
    theta = np.linalg.pinv(X_train_b) @ y_train

    # Predict on validation
    y_hat = X_val_b @ theta

    # RSS and sqrt(RSS)
    rss = np.sum((y_val - y_hat) ** 2)
    sqrt_rss = np.sqrt(rss)
    sqrt_rss_per_fold.append(sqrt_rss)

    print(f"Fold {k+1}: sqrt_RSS = {sqrt_rss:.4f}")

avg_sqrt_rss = np.mean(sqrt_rss_per_fold)
print(f"\nAverage sqrt_RSS over 5 folds = {avg_sqrt_rss:.4f}")


Fold 1: sqrt_RSS = 621.6119
Fold 2: sqrt_RSS = 772.9777
Fold 3: sqrt_RSS = 555.7166
Fold 4: sqrt_RSS = 1377.2311
Fold 5: sqrt_RSS = 1530.0825

Average sqrt_RSS over 5 folds = 971.5239


At the end in this part, please repeat the same experiment as in the previous step, but instead of linear regression, implement linear regression **with L2-norm regularization**. Experiment and report your results (average square root of RSS over 5-fold cross-validation) with different values of the regularization term $\lambda=\{1, 0.1, 0.01, 0.001, 0.0001\}$.

**Hint: details about the closed-form solution with regularization is on page 74 in slides of our linear regression lecture.**

In [ ]:
# Write your code here
# --- RIDGE (L2) LINEAR REGRESSION WITH 5-FOLD CROSS-VALIDATION (no sklearn) ---
# Assumes:
#   - `features` is your final feature matrix (one-hot done; numeric optionally standardized)
#   - `labels` is your target vector
# What this does:
#   - Repeats the OLS experiment but with L2 regularization (Ridge)
#   - Tests multiple lambda values and reports sqrt_RSS for each fold + average

import numpy as np

X = features.astype(float)
y = labels.astype(float).ravel()  # ensure 1D

# Helper: add a bias (intercept) column of ones
def add_bias(A):
    return np.hstack([np.ones((A.shape[0], 1)), A])

# Ridge closed-form solution:
# theta = (X^T X + λ * R)^(-1) X^T y
# where R is identity *except* we do NOT regularize the bias term (R[0,0] = 0)
def ridge_fit(Xb, y, lam):
    d = Xb.shape[1]
    R = np.eye(d)
    R[0, 0] = 0.0  # do not regularize bias
    XtX = Xb.T @ Xb
    Xty = Xb.T @ y
    theta = np.linalg.pinv(XtX + lam * R) @ Xty
    return theta

def sqrt_rss(y_true, y_pred):
    return np.sqrt(np.sum((y_true - y_pred) ** 2))

# Make 5 folds (shuffled, reproducible)
rng = np.random.default_rng(42)
idx = np.arange(X.shape[0])
rng.shuffle(idx)
folds = np.array_split(idx, 5)

# Add bias once
Xb = add_bias(X)

lambdas = [1, 0.1, 0.01, 0.001, 0.0001]

for lam in lambdas:
    sqrt_rss_per_fold = []
    print(f"\n=== λ = {lam} ===")
    for k in range(5):
        val_idx = folds[k]
        train_idx = np.concatenate([folds[i] for i in range(5) if i != k])

        X_train_b, y_train = Xb[train_idx], y[train_idx]
        X_val_b,   y_val   = Xb[val_idx],   y[val_idx]

        # Fit ridge model and predict
        theta = ridge_fit(X_train_b, y_train, lam)
        y_hat = X_val_b @ theta

        # Compute sqrt(RSS) for this fold
        s = sqrt_rss(y_val, y_hat)
        sqrt_rss_per_fold.append(s)
        print(f"Fold {k+1}: sqrt_RSS = {s:.4f}")

    avg_sqrt_rss = np.mean(sqrt_rss_per_fold)
    print(f"Average sqrt_RSS over 5 folds (λ={lam}) = {avg_sqrt_rss:.4f}")



=== λ = 1 ===
Fold 1: sqrt_RSS = 596.1611
Fold 2: sqrt_RSS = 766.0234
Fold 3: sqrt_RSS = 526.5059
Fold 4: sqrt_RSS = 1140.9784
Fold 5: sqrt_RSS = 1527.3252
Average sqrt_RSS over 5 folds (λ=1) = 911.3988

=== λ = 0.1 ===
Fold 1: sqrt_RSS = 617.9304
Fold 2: sqrt_RSS = 772.1057
Fold 3: sqrt_RSS = 551.7590
Fold 4: sqrt_RSS = 1343.3902
Fold 5: sqrt_RSS = 1529.5176
Average sqrt_RSS over 5 folds (λ=0.1) = 962.9406

=== λ = 0.01 ===
Fold 1: sqrt_RSS = 621.2279
Fold 2: sqrt_RSS = 772.8880
Fold 3: sqrt_RSS = 555.3049
Fold 4: sqrt_RSS = 1374.1429
Fold 5: sqrt_RSS = 1529.8394
Average sqrt_RSS over 5 folds (λ=0.01) = 970.6806

=== λ = 0.001 ===
Fold 1: sqrt_RSS = 621.5733
Fold 2: sqrt_RSS = 772.9687
Fold 3: sqrt_RSS = 555.6753
Fold 4: sqrt_RSS = 1377.3758
Fold 5: sqrt_RSS = 1529.8729
Average sqrt_RSS over 5 folds (λ=0.001) = 971.4932

=== λ = 0.0001 ===
Fold 1: sqrt_RSS = 621.6080
Fold 2: sqrt_RSS = 772.9768
Fold 3: sqrt_RSS = 555.7125
Fold 4: sqrt_RSS = 1377.7007
Fold 5: sqrt_RSS = 1529.8763
Aver

In [ ]:
#optional
# --- TRAIN FINAL RIDGE MODEL USING ALL DATA (λ = 1) ---
X = features.astype(float)
y = labels.astype(float).ravel()

def add_bias(A):
    return np.hstack([np.ones((A.shape[0], 1)), A])

def ridge_fit(Xb, y, lam):
    d = Xb.shape[1]
    R = np.eye(d)
    R[0, 0] = 0.0  # don't regularize bias
    XtX = Xb.T @ Xb
    Xty = Xb.T @ y
    theta = np.linalg.pinv(XtX + lam * R) @ Xty
    return theta

# Add bias and fit model
Xb = add_bias(X)
lambda_best = 1
theta_final = ridge_fit(Xb, y, lambda_best)

print("Model trained successfully! Shape of weights:", theta_final.shape)



Model trained successfully! Shape of weights: (22,)


In [ ]:
#optional
# --- FUNCTION TO PREDICT COMBAT POINT FOR A NEW POKEMON ---

def predict_combat_point(stamina, attack, defense, capture_rate, flee_rate, spawn_chance, primary_strength):
    # Numeric features (same order as training)
    numeric_values = np.array([stamina, attack, defense, capture_rate, flee_rate, spawn_chance], dtype=float)

    # Standardize numeric values using training means and stds
    numeric_standardized = (numeric_values - mean) / std

    # One-hot encode the primary_strength using the same categories
    one_hot = np.zeros(len(unique_strengths))
    if primary_strength in unique_strengths:
        idx = np.where(unique_strengths == primary_strength)[0][0]
        one_hot[idx] = 1.0
    else:
        print(f"⚠️ Unknown strength '{primary_strength}'. Valid options are: {unique_strengths}")

    # Combine numeric + one-hot
    features_new = np.hstack([numeric_standardized, one_hot])

    # Add bias
    features_new_b = np.hstack([1.0, features_new])

    # Predict combat point
    predicted_cp = features_new_b @ theta_final
    return predicted_cp
# Example 1: Fire-type Pokémon
pred1 = predict_combat_point(120, 150, 100, 0.4, 0.1, 0.8, "Fire")
print(f"🔥 Predicted Combat Point (Fire-type) = {pred1:.2f}")

# Example 2: Water-type Pokémon
pred2 = predict_combat_point(130, 140, 110, 0.5, 0.2, 0.7, "Water")
print(f"💧 Predicted Combat Point (Water-type) = {pred2:.2f}")

# Example 3: Grass-type Pokémon
pred3 = predict_combat_point(100, 120, 80, 0.6, 0.3, 0.9, "Grass")
print(f"🌿 Predicted Combat Point (Grass-type) = {pred3:.2f}")



🔥 Predicted Combat Point (Fire-type) = 1304.83
💧 Predicted Combat Point (Water-type) = 1353.23
🌿 Predicted Combat Point (Grass-type) = 846.35


## Part 2: Neural Networks (40 points)

In this part, you are going to implement your multi-layer perceptron model by the Pytorch library. You will still use the same handwritten digit image dataset from HW1. So, in the next few cells, please run the provided code to load and process the data, and create dataset objects for further use by Pytorch.

**Note: you need to install Pytorch beforehand (check out https://pytorch.org/get-started/locally/). Or, you can use Google Colab for this homework, which is recommended.**

In [ ]:
# load data from file and split into training and validation sets
import numpy as np
data = np.loadtxt("/content/drive/MyDrive/hw2/train.txt", delimiter=',')
perm_idx = np.random.permutation(data.shape[0])
vali_num = int(data.shape[0] * 0.2)
vali_idx = perm_idx[:vali_num]
train_idx = perm_idx[vali_num:]
train_data = data[train_idx]
vali_data = data[vali_idx]
train_features = train_data[:, 1:].astype(np.float32)
train_labels = train_data[:, 0].astype(int)
vali_features = vali_data[:, 1:].astype(np.float32)
vali_labels = vali_data[:, 0].astype(int)

In [ ]:
# define a Dataset class
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class MNISTDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx, :], self.labels[idx]

In [ ]:
training_data = MNISTDataset(train_features, train_labels)
vali_data = MNISTDataset(vali_features, vali_labels)
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
vali_dataloader = DataLoader(vali_data, batch_size=batch_size)

for X, y in train_dataloader:
    print(f"Shape of X [N, F]: {X.shape} {X.dtype}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, F]: torch.Size([64, 784]) torch.float32
Shape of y: torch.Size([64]) torch.int64


Now, you should have the train_dataloader and vali_dataloader. Then, you need to build and train your multi-layer perceptron model by Pytorch.

https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html gives a comprehansive example how to achieve this. Please read this tutorial closely, and implement the model in the next few cells.

https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/c30c1dcf2bc20119bcda7e734ce0eb42/quickstart_tutorial.ipynb provides the interactive version, which you can run and edit.

**Note: in your implementation:**
- you will only have three layers [784 -> 512 -> 10], you need to remove the [512 -> 512] layer in the tutorial.
- add 'weight_decay=1e-4' in torch.optim.SGD to add L2 regularization.
- train the model for 10 epochs instead of 5 epochs.
- keep all other hyper-parameters the same as used in the tutorial.
- **You are allowed to resue the code in the tutorial for this homework**


**Note: print out the training process and the final accuracy on the validation set.**

**Note: you can use Colab for running the code with GPU for free (open a colab notebook, then Runtime->Change runtime type->Hardware accelerator->GPU)**

In [ ]:
# Write your code

# --- PART 2: Multi-Layer Perceptron (784 -> 512 -> 10) using PyTorch ---
# HW2 Data Mining :: CS 584 :: GMU
# You must have train_dataloader and vali_dataloader ready before running this cell.

import torch
from torch import nn
from torch.utils.data import DataLoader

# Set random seed and device
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -------------------------
# Step 1: Define the MLP model
# -------------------------
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Flatten ensures the input (64, 784) remains consistent
        self.flatten = nn.Flatten()
        # Only three layers: [784 -> 512 -> 10]
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# Instantiate model
model = MLP().to(device)
print(model)

# -------------------------
# Step 2: Define Loss Function & Optimizer
# -------------------------
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
# add weight_decay = 1e-4 for L2 regularization
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4)

# -------------------------
# Step 3: Training and Evaluation Loops
# -------------------------
def train(dataloader, model, loss_fn, optimizer, epoch):
    model.train()
    size = len(dataloader.dataset)
    running_loss = 0.0
    for batch, (X, y) in enumerate(dataloader, start=1):
        X, y = X.to(device), y.to(device)

        # Forward pass
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch % 100 == 0 or batch == len(dataloader):
            avg_loss = running_loss / batch
            print(f"Epoch {epoch:02d} | Batch {batch:4d}/{len(dataloader)} | Loss: {avg_loss:.4f}")

def evaluate(dataloader, model, loss_fn):
    model.eval()
    num_batches = len(dataloader)
    total_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            total_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            total += y.size(0)
    avg_loss = total_loss / num_batches
    accuracy = correct / total
    return avg_loss, accuracy

# -------------------------
# Step 4: Train for 10 epochs
# -------------------------
epochs = 10
for t in range(1, epochs + 1):
    train(train_dataloader, model, loss_fn, optimizer, t)
    val_loss, val_acc = evaluate(vali_dataloader, model, loss_fn)
    print(f"Epoch {t:02d} complete. Validation Loss: {val_loss:.4f} | Validation Accuracy: {val_acc*100:.2f}%")

print("\n✅ Final Validation Accuracy:", f"{val_acc*100:.2f}%")


Using device: cpu
MLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
  )
)
Epoch 01 | Batch  100/750 | Loss: 3.9878
Epoch 01 | Batch  200/750 | Loss: 2.3422
Epoch 01 | Batch  300/750 | Loss: 1.7108
Epoch 01 | Batch  400/750 | Loss: 1.3825
Epoch 01 | Batch  500/750 | Loss: 1.1706
Epoch 01 | Batch  600/750 | Loss: 1.0287
Epoch 01 | Batch  700/750 | Loss: 0.9215
Epoch 01 | Batch  750/750 | Loss: 0.8764
Epoch 01 complete. Validation Loss: 0.2686 | Validation Accuracy: 93.72%
Epoch 02 | Batch  100/750 | Loss: 0.1742
Epoch 02 | Batch  200/750 | Loss: 0.1739
Epoch 02 | Batch  300/750 | Loss: 0.1593
Epoch 02 | Batch  400/750 | Loss: 0.1562
Epoch 02 | Batch  500/750 | Loss: 0.1523
Epoch 02 | Batch  600/750 | Loss: 0.1506
Epoch 02 | Batch  700/750 | Loss: 0.1473
Epoch 02 | Batch  750/750 | Loss: 0.1456
Epoch 02 complete. Val

## Part 3: Tune Hyperparameter (20 points)

In this part, you need to do your best to tune the hyperparameters in the MLP to build the best model and evaluate the predictions for the testing data.

Now, you should tune four hyperparameters:

- the number of layers and the dimension of each layer (explore as much as you can, but choose reasonable settings considering the computational resources you have)
- the activation function (choose from sigmoid, tanh, relu)
- weight decay
- number of training epochs


**Hint: You can tune these hyperparameters by one randomly generated validation set, or you can also use the cross-validation method.**

**Note: you can use Colab for running the code with GPU for free**


In [13]:
# Write your code here
# --- PART 3: Hyperparameter Tuning for MLP (PyTorch) ---
# Tries multiple architectures, activations, weight_decay values, and epoch counts.
# Reports the best validation accuracy and a summary table of all runs.

import torch
from torch import nn
import itertools
import time
import pandas as pd

# Repro + device
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------- Search Space ----------
# Hidden layer architectures: each list is the sequence of hidden sizes
architectures = [
    [256],
    [512],
    [512, 256],
    [512, 512],
    [256, 256],
]

activations = {
    "relu": nn.ReLU,
    "tanh": nn.Tanh,
    "sigmoid": nn.Sigmoid,
}

weight_decays = [0.0, 1e-5, 1e-4, 1e-3]
epoch_choices = [5, 10, 15]  # try short, medium, longer training

# Keep other tutorial hyperparams the same
learning_rate = 1e-3
criterion = nn.CrossEntropyLoss()

# ---------- Helpers ----------
class FlexibleMLP(nn.Module):
    """
    Builds an MLP: 784 -> (hidden layers) -> 10
    Activation placed between hidden layers only.
    """
    def __init__(self, hidden_layers, activation_name="relu"):
        super().__init__()
        act = activations[activation_name]  # class, not instance
        layers = []
        in_dim = 784
        for h in hidden_layers:
            layers.append(nn.Linear(in_dim, h))
            layers.append(act())
            in_dim = h
        layers.append(nn.Linear(in_dim, 10))
        self.net = nn.Sequential(nn.Flatten(), *layers)

    def forward(self, x):
        return self.net(x)

def train_one_model(model, train_loader, val_loader, epochs, weight_decay):
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    def evaluate(loader):
        model.eval()
        total, correct, total_loss = 0, 0, 0.0
        with torch.no_grad():
            for X, y in loader:
                X, y = X.to(device), y.to(device)
                logits = model(X)
                loss = criterion(logits, y).item()
                total_loss += loss
                preds = logits.argmax(1)
                correct += (preds == y).sum().item()
                total += y.size(0)
        return total_loss / max(1, len(loader)), correct / max(1, total)

    # Train
    for ep in range(1, epochs + 1):
        model.train()
        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            logits = model(X)
            loss = criterion(logits, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Final validation after last epoch
    val_loss, val_acc = evaluate(val_loader)
    return val_loss, val_acc, model

# ---------- Grid/Random Search (here: grid over all combinations) ----------
results = []
best = {"val_acc": -1.0}

combo_iter = list(itertools.product(architectures, activations.keys(), weight_decays, epoch_choices))
print(f"Total runs: {len(combo_iter)}")

start_all = time.time()
for i, (arch, act, wd, ep) in enumerate(combo_iter, 1):
    start = time.time()
    model = FlexibleMLP(arch, act)
    val_loss, val_acc, trained_model = train_one_model(model, train_dataloader, vali_dataloader, ep, wd)
    dur = time.time() - start

    results.append({
        "run": i,
        "arch": str(arch),
        "activation": act,
        "weight_decay": wd,
        "epochs": ep,
        "val_loss": float(val_loss),
        "val_acc": float(val_acc),
        "seconds": round(dur, 2),
    })

    if val_acc > best["val_acc"]:
        best.update({
            "val_acc": val_acc,
            "val_loss": val_loss,
            "arch": arch,
            "activation": act,
            "weight_decay": wd,
            "epochs": ep,
            "model_state": trained_model.state_dict(),  # save best weights
        })

    print(f"[{i}/{len(combo_iter)}] arch={arch}, act={act}, wd={wd}, ep={ep} "
          f"=> ValAcc={val_acc*100:.2f}% (loss={val_loss:.4f})")

total_time = time.time() - start_all
print(f"\nSearch finished in {total_time:.1f} sec")

summary_df = pd.DataFrame(results).sort_values(by="val_acc", ascending=False).reset_index(drop=True)
print("\n=== Top 10 runs by validation accuracy ===")
print(summary_df.head(10))

print("\n=== Best configuration ===")
print({
    "arch": best["arch"],
    "activation": best["activation"],
    "weight_decay": best["weight_decay"],
    "epochs": best["epochs"],
    "val_acc": round(best["val_acc"] * 100, 2),
    "val_loss": round(best["val_loss"], 4),
})

# (Optional) Rebuild and load best model for later test-time evaluation:
best_model = FlexibleMLP(best["arch"], best["activation"]).to(device)
best_model.load_state_dict(best["model_state"])
best_model.eval()


Using device: cpu
Total runs: 180
[1/180] arch=[256], act=relu, wd=0.0, ep=5 => ValAcc=95.26% (loss=0.1794)
[2/180] arch=[256], act=relu, wd=0.0, ep=10 => ValAcc=96.17% (loss=0.1515)
[3/180] arch=[256], act=relu, wd=0.0, ep=15 => ValAcc=96.14% (loss=0.1683)
[4/180] arch=[256], act=relu, wd=1e-05, ep=5 => ValAcc=95.06% (loss=0.1812)
[5/180] arch=[256], act=relu, wd=1e-05, ep=10 => ValAcc=95.86% (loss=0.1672)
[6/180] arch=[256], act=relu, wd=1e-05, ep=15 => ValAcc=96.00% (loss=0.1727)
[7/180] arch=[256], act=relu, wd=0.0001, ep=5 => ValAcc=95.30% (loss=0.1802)
[8/180] arch=[256], act=relu, wd=0.0001, ep=10 => ValAcc=96.04% (loss=0.1712)
[9/180] arch=[256], act=relu, wd=0.0001, ep=15 => ValAcc=96.22% (loss=0.1711)
[10/180] arch=[256], act=relu, wd=0.001, ep=5 => ValAcc=95.33% (loss=0.1768)
[11/180] arch=[256], act=relu, wd=0.001, ep=10 => ValAcc=95.95% (loss=0.1674)
[12/180] arch=[256], act=relu, wd=0.001, ep=15 => ValAcc=95.91% (loss=0.1660)
[13/180] arch=[256], act=tanh, wd=0.0, ep=5 =>

FlexibleMLP(
  (net): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): ReLU()
    (5): Linear(in_features=512, out_features=10, bias=True)
  )
)

### Question: What is your final hyperparameter setting? How do you tune them? What choices have you tried?

 To find the best model, I experimented with different hyperparameters — including the number of layers, activation functions, weight decay values, and training epochs.
I tested architectures like [256], [512], [512, 256], [512, 512], and [256, 256] using activations (ReLU, tanh, sigmoid), weight decay (0, 1e-5, 1e-4, 1e-3), and epochs (5, 10, 15).
After evaluating 180 runs, the best configuration was [512, 512] layers with ReLU, weight_decay = 0.0001, and 15 epochs, achieving about 96.9 % validation accuracy.
ReLU consistently performed the best because it avoids vanishing gradients, and a small weight decay helped reduce overfitting for stable training.

Please apply the model with the best hyperparameter setting you find above to the testing set. First of all, let's load the testing data by executing the following code.

In [14]:
test_features = np.loadtxt("/content/drive/MyDrive/hw2/test.txt", delimiter=',')
test_labels = np.loadtxt("/content/drive/MyDrive/hw2/test_label.txt")
print('array of testing feature matrix: shape ' + str(np.shape(test_features)))
print('array of testing labels: shape ' + str(np.shape(test_labels)))

array of testing feature matrix: shape (10000, 784)
array of testing labels: shape (10000,)


Now, report the Accuracy of your best model on the testing set.

**Hint: use the following two lines of code to generate the label predictions for test data:**
- raw_pred = model(torch.tensor(test_features).to(device).float())
- pred = np.argmax(raw_pred.to('cpu').detach().numpy(), axis=1)

In [18]:
# ✅ Final test evaluation code

# 1) Ensure test labels are integer
test_labels = test_labels.astype(int)

# 2) Load the best model
try:
    best_model.eval()
    used_model = best_model
    used_arch = best["arch"]
    used_act  = best["activation"]
except NameError:
    try:
        used_arch = best["arch"]
        used_act  = best["activation"]
        used_model = FlexibleMLP(used_arch, used_act).to(device)
        used_model.load_state_dict(best["model_state"])
        used_model.eval()
    except Exception as e:
        raise RuntimeError("Best model not found. Please re-run the tuning cell first.") from e

print(f"\nUsing best model: arch={used_arch}, activation={used_act}")

# 3) Forward pass on test set
with torch.no_grad():
    raw_pred = used_model(torch.tensor(test_features).to(device).float())
    pred = np.argmax(raw_pred.to('cpu').detach().numpy(), axis=1)

# 4) Compute accuracy
test_accuracy = (pred == test_labels).mean() * 100.0
print(f"\n✅ Test Accuracy of best model: {test_accuracy:.2f}%")



Using best model: arch=[512, 512], activation=relu

✅ Test Accuracy of best model: 96.90%
